In [12]:
import pandas as pd
import numpy as np
import time

In [2]:
crm = pd.read_excel(r'C:\Users\Mironov1-AM\PycharmProjects\Офис\Gifts\CRM_cut.xlsx')
tmc = pd.read_excel(r'C:\Users\Mironov1-AM\PycharmProjects\Офис\Gifts\TMC.xlsx')

# crm = crm.head(5)

In [3]:
crm = crm[['GIFT_NAME1', 'GIFT_SUM1']]
crm['GIFT_NAME1'] = crm['GIFT_NAME1'].str.lower()
crm['GIFT_NAME1'] = crm['GIFT_NAME1'].str.strip()

tmc['find_text'] = tmc['find_text'].str.lower()
tmc['find_text'] = tmc['find_text'].str.strip()
tmc['exclude_text'] = tmc['exclude_text'].str.lower()
tmc['exclude_text'] = tmc['exclude_text'].str.strip()

crm['GIFT_NAME1'] = crm['GIFT_NAME1'].map(lambda x: str(x).replace("ё", "е")).astype(str)
tmc['find_text'] = tmc['find_text'].map(lambda x: str(x).replace("ё", "е")).astype(str)
tmc['exclude_text'] = tmc['exclude_text'].map(lambda x: str(x).replace("ё", "е")).astype(str)

MONTHS = ['янв', 'фев', "мар", "апр", "май", "июн", "июл", "авг", "сен", "окт", "ноя", "дек"]

stop_words = [',', '.']

for word in stop_words:
    crm['GIFT_NAME1'] = crm['GIFT_NAME1'].str.replace(word, '', regex=False)
    tmc['find_text'] = tmc['find_text'].str.replace(word, '', regex=False)


In [4]:
def levenshtein(seq1, seq2):
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros((size_x, size_y))
    for x in range(size_x):
        matrix[x, 0] = x
    for y in range(size_y):
        matrix[0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x - 1] == seq2[y - 1]:
                matrix[x, y] = min(
                    matrix[x - 1, y] + 1,
                    matrix[x - 1, y - 1],
                    matrix[x, y - 1] + 1
                )
            else:
                matrix[x, y] = min(
                    matrix[x - 1, y] + 1,
                    matrix[x - 1, y - 1] + 1,
                    matrix[x, y - 1] + 1
                )
    return (matrix[size_x - 1, size_y - 1])

In [5]:
def change_sum_value(x):
    if type(x) != str:
        return x
    if x[-3:] in MONTHS:
        x = x[:x.find('.')] + ',' + str(MONTHS.index(x[-3:]) + 1)

    result = x.replace(",", ".")
    if result.replace('.', '').isdigit():
        result = float(result)

    return result

In [6]:
crm['GIFT_SUM1'] = crm['GIFT_SUM1'].apply(change_sum_value)

ii = 0
number_loading = len(crm['GIFT_NAME1'])

In [9]:
def func_find_tmc(x: dict, is_levenshtein=False):
    global ii
    global number_loading
    crm_text = x['GIFT_NAME1']
    crm_price = x['GIFT_SUM1']
    crm_words = crm_text.split()

    if type(x['GIFT_SUM1']) == str or pd.isnull(x['GIFT_SUM1']) or pd.isnull(x['GIFT_NAME1']):
        return [-1, '']

    tmp = tmc[tmc['price'] == -1][['price', 'short_name']]

    for i in range(tmc.shape[0]):
        is_find = True
        find_text = tmc.loc[i, 'find_text'].split()
        
        # =====================
        for text in find_text:
            if text not in crm_text:
                is_find = False
                break
        if not is_levenshtein or is_find:
            continue
        
        is_find = True
        for text in find_text:
            is_find_one_word = False
            for crm_word in crm_words:
                leven = levenshtein(crm_word, text)
                if leven <= 1:
                    is_find_one_word = True
                    break
            is_find = is_find and is_find_one_word
            if not is_find:
                break        
                
        if not pd.isnull(tmc.loc[i, 'exclude_text']):
            exclude_text = tmc.loc[i, 'exclude_text'].split()
            for text in exclude_text:
                if text in crm_text:
                    is_find = False
                    break

        if not is_find:
            continue
        

        tmp = tmp.append(tmc.loc[i, ['price', 'short_name']], ignore_index=True)
        print(tmp)
    tmp['ones'] = 1
    tmp = tmp.reset_index()

    merge_tmc = list(tmp['short_name'].unique())
    if len(merge_tmc) == 0:
        return [-1, '']

    merged_df = tmp[tmp['short_name'] == merge_tmc[0]].reset_index()

    for j, word in enumerate(merge_tmc[1:]):
        try:
            merged_df = merged_df.merge(
                tmp[tmp['short_name'] == word],
                on='ones',
                how='outer',
                suffixes=['_' + str(j), '_' + str(j + 1)]
            )
        except MemoryError:
            print(ii)
            return [-1, '']

    merged_df['total_cost'] = 0
    merged_df['all_tmc'] = ''
    for column in merged_df.columns:
        if 'price' in column:
            merged_df['total_cost'] += merged_df[column]
        if 'short_name' in column:
            merged_df['all_tmc'] += '|' + merged_df[column]

    merged_df['different_price'] = abs(merged_df['total_cost'] - crm_price)
    result = merged_df.sort_values('different_price').reset_index(drop=True)
    del merged_df
    del tmp

    if len(result) == 0:
        return [-1, '']

    result = result.loc[0, ['total_cost', 'all_tmc']]
    PRINT(ii)
    ii += 1
    loading = ii/number_loading*100
    loading = int(loading)
    print(f'\r [{"█" * loading}{" " * (100 - loading)}] - {loading * 1}%', end="")

    return pd.Series(result)

In [10]:
st_t = time.time()
crm[['total_cost', 'all_tmc']] = crm.apply(lambda x: func_find_tmc(x, is_levenshtein=True), axis=1)
# crm.to_excel('result_Gifts2.xlsx', index=False)
print("")
print(crm)
print(time.time() - st_t)


   price short_name
0  126.0    Шоколад
   price short_name
0  126.0    Шоколад
1  142.8    Шоколад
   price short_name
0  126.0    Шоколад
1  142.8    Шоколад
2   98.4      Ручка
   price short_name
0  126.0    Шоколад
1  142.8    Шоколад
2   98.4      Ручка
3  130.8      Ручка
   price short_name
0  126.0    Шоколад
1  142.8    Шоколад
2   98.4      Ручка
3  130.8      Ручка
4  210.0      Ручка
   price short_name
0  126.0    Шоколад
1  142.8    Шоколад
2   98.4      Ручка
3  130.8      Ручка
4  210.0      Ручка
5  170.4      Ручка
   price short_name
0  126.0    Шоколад
1  142.8    Шоколад
2   98.4      Ручка
3  130.8      Ручка
4  210.0      Ручка
5  170.4      Ручка
6  118.8      Ручка
   price short_name
0  126.0    Шоколад
1  142.8    Шоколад
2   98.4      Ручка
3  130.8      Ручка
4  210.0      Ручка
5  170.4      Ручка
6  118.8      Ручка
7   46.8      Ручка
   price short_name
0  126.0    Шоколад
1  142.8    Шоколад
2   98.4      Ручка
3  130.8      Ручка
4  210.0      Ручка


     price short_name
0    126.0    Шоколад
1    142.8    Шоколад
2     98.4      Ручка
3    130.8      Ручка
4    210.0      Ручка
5    170.4      Ручка
6    118.8      Ручка
7     46.8      Ручка
8     48.0      Ручка
9     39.6      Ручка
10   120.0      Ручка
11    97.2      Ручка
12    36.0      Ручка
13   162.0    Шоколад
14   148.8    Шоколад
15   187.2    Шоколад
16   417.6    Шоколад
17   112.8    Шоколад
18  1413.6    Шоколад
19   696.0    Шоколад
20   438.0    Шоколад
21   720.0    Шоколад
22  1368.0    Шоколад
23   465.6    Шоколад
24   396.0    Шоколад
25   534.0    Шоколад
26   213.6    Шоколад
27   218.4    Шоколад
28   393.6    Шоколад
29   787.2    Шоколад
30   716.4    Шоколад
31  1357.2    Шоколад
32   324.0    Шоколад
33   452.4    Шоколад
34  1404.0    Шоколад
     price short_name
0    126.0    Шоколад
1    142.8    Шоколад
2     98.4      Ручка
3    130.8      Ручка
4    210.0      Ручка
5    170.4      Ручка
6    118.8      Ручка
7     46.8      Ручка
8     48.0

NameError: ("name 'PRINT' is not defined", 'occurred at index 1')

In [ ]:
levenshtein('ручек', 'ручка')

In [ ]:
tmc['find_text'].unique()

In [ ]:
tmc_word = 'ручка'

stroka = 'подариларшоколадиручку'

In [ ]:
j=0
z=0
for i in stroka:
    if i in tmc_word[z]:
        j +=1
        z +=1
        if i == tmc_word[0]:
            j = 1
    else:
        z = 0  
    if len(tmc_word) == j:
        break
j

In [ ]:
len(tmc_word)

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024 ** 2
    print('Использование памяти DataFrame {:.2f} Mb'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('Использовании памяти после оптимизации: {:.2f} Mb'.format(end_mem))
    print('Уменьшился на {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df


In [ ]:
reduce_mem_usage(tmc)

In [ ]:
def levenshtein(seq1, seq2):
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros((size_x, size_y))
    for x in range(size_x):
        matrix[x, 0] = x
    for y in range(size_y):
        matrix[0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x - 1] == seq2[y - 1]:
                matrix[x, y] = min(
                    matrix[x - 1, y] + 1,
                    matrix[x - 1, y - 1],
                    matrix[x, y - 1] + 1
                )
            else:
                matrix[x, y] = min(
                    matrix[x - 1, y] + 1,
                    matrix[x - 1, y - 1] + 1,
                    matrix[x, y - 1] + 1
                )
    return (matrix[size_x - 1, size_y - 1])

In [ ]:
def main_series(x: dict, tmc, is_levenshtein=False):
    crm_text = x['GIFT_NAME1']
    crm_words = crm_text.split()
    for i in range(tmc.shape[0]):
        is_find = True
        find_text = tmc.loc[i, 'find_text'].split()
        for text in find_text:
            if text not in crm_text:
                if not is_levenshtein:
                    is_find = False
                    break
                for crm_word in crm_words:
                    leven = levenshtein(crm_word, text)
                    if leven <= 1:
                        is_find = True
                        break
                    else:
                        is_find = False
        if not pd.isnull(tmc.loc[i, 'exclude_text']):
            exclude_text = tmc.loc[i, 'exclude_text'].split()
            for text in exclude_text:
                if text in crm_text:
                    is_find = False
                    break
        is_find = is_find
        if not is_find:
            continue

    return is_find

In [ ]:
crm['is_find'] = crm.apply(lambda x: main_series(x, tmc, is_levenshtein=True), axis=1)

In [ ]:
crm

In [38]:
x = [1,2,3,4,5,6]
ii = 0
number_loading_ii = len(x)
def xxx(list):
    global ii
    global number_loading_ii
    y = []
    for i in list:
        y.append(i)
        ii += 1
        loading_ii = ii / number_loading_ii * 100.00
        loading_ii = int(loading_ii)
        print(f'\r[{"█" * loading_ii}{" " * (100 - loading_ii)}] - {loading_ii * 1}%', end="")
        time.sleep(1)
    return y

mylist = xxx(x)


[████████████████████████████████████████████████████████████████████████████████████████████████████] - 100%